In [1]:
import comet_ml
import torch
from datasets import load_from_disk

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenized_dataset = load_from_disk('/shared/Tony/MSc2024/data/tokenized_preprocessed_wikitext103.hf') #20231101
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]

In [3]:
#comet_ml.init(project_name="distilbert_dotprod")
from transformers import AutoTokenizer
import random

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
num_samples = len(train_dataset)//10
train_subset = train_dataset.select(range(num_samples))
print(train_subset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 85965
})


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments, TrainerCallback
import matplotlib.pyplot as plt
import numpy as np
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

class distillTrainer(Trainer):
    def __init__(self, *args, teacher_model = None, hidden = False, **kwargs):
        super().__init__(*args,**kwargs)
        self.teacher = teacher_model
        self.teacher.eval()
        self.layer_groups = [f"transformer.layer.{i}" for i in range(6)] 
        self.current_layer_group = 0
        self.unfrozen_layers = set()
        self.layer_logs = []
        self.context_loss_stats = []
        self.hidden = hidden
        self.cosine_loss_fct = nn.CosineEmbeddingLoss(reduction="mean")

    def hidden_state_loss(self, student_outputs, teacher_outputs):
        s_hidden_states = student_outputs
        t_hidden_states = teacher_outputs
        assert t_hidden_states.size() == s_hidden_states.size()
        dim = s_hidden_states.size(-1)
        s_hidden_states_slct = s_hidden_states.view(-1, dim)
        t_hidden_states_slct = t_hidden_states.view(-1, dim)

        target = s_hidden_states_slct.new(s_hidden_states_slct.size(0)).fill_(1) 
        loss = self.cosine_loss_fct(s_hidden_states_slct, t_hidden_states_slct, target)
        #loss = F.mse_loss(s_hidden_states_slct, t_hidden_states_slct)
        return loss
        
    def compute_loss(self, model, inputs, return_outputs = False):
        student_outputs = model(**inputs)
        student_logits = student_outputs.logits

        with torch.no_grad():
            teacher_outputs = self.teacher(**inputs)
            teacher_logits = teacher_outputs.logits

        student_obj = student_outputs.hidden_states[self.current_layer_group + 1] # +1 because idx 0 is the embedding layer.
        teacher_obj = teacher_outputs.hidden_states[self.current_layer_group + 1]


        loss = self.hidden_state_loss(student_obj,teacher_obj)
      
        #student_obj = student_outputs.contexts[self.current_layer_group]
        #teacher_obj = teacher_outputs.contexts[self.current_layer_group]
        #loss = F.mse_loss(student_obj, teacher_obj)

        student_stats = {
            'mean': student_obj.mean().item(),
            'max': student_obj.max().item(),
            'min': student_obj.min().item()
        }
        teacher_stats = {
            'mean': teacher_obj.mean().item(),
            'max': teacher_obj.max().item(),
            'min': teacher_obj.min().item()
        }

        self.context_loss_stats.append({
            'layer': self.current_layer_group,
            'student': student_stats,
            'teacher': teacher_stats,
        })
        
        return (loss, student_outputs) if return_outputs else loss
        
    def train(self, resume_from_checkpoint=None, **kwargs):
        layer_plots = []
        for layer_group in self.layer_groups:
            print(f"Training layer group: {layer_group}")
            self.switch_to_next_layer_group()
            print(self.get_num_trainable_parameters())
            res = super().train(resume_from_checkpoint=resume_from_checkpoint, **kwargs)
            self.layer_logs.append(self.state.log_history.copy())
            self.current_layer_group += 1
            #self.save_model(f"./results/layer_{layer_group}")
        self.plot_layer_losses()
        return res

    def freeze_all_layers(self):
        for param in self.model.parameters():
            param.requires_grad = False
    
    def switch_to_next_layer_group(self):
        self.freeze_all_layers()
        print("Current layer", self.current_layer_group)
        
        if self.current_layer_group < len(self.layer_groups):
            current_layer = self.layer_groups[self.current_layer_group]
            newly_unfrozen_params = []
            
            for name, param in self.model.named_parameters():
                if current_layer in name: #and any(qkv in name for qkv in ['q_lin', 'k_lin', 'v_lin', 'out_lin', 'sa_layer_norm','ffn','output_layer_norm']): #v_lin q_lin k_lin
                    param.requires_grad = True
                    newly_unfrozen_params.append(param)
                        
            print(f"Unfrozen parameters for layer {self.current_layer_group}:")
            for name, param in self.model.named_parameters():
                if param.requires_grad:
                    print(f"  - {name}")

            optimizer_grouped_parameters = [
            {
                "params": [p for p in newly_unfrozen_params if p.requires_grad],
                "weight_decay": self.args.weight_decay,
            }
            ]
            
            self.optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate)
            
            num_training_steps = len(self.train_dataset) // self.args.train_batch_size * self.args.num_train_epochs
            warmup_rate = 0.05  # 10% 
            warmup_steps = int(num_training_steps * warmup_rate)
            self.lr_scheduler = get_cosine_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=warmup_steps, 
                num_training_steps=num_training_steps
            )

    def plot_layer_losses(self):
        fig, axs = plt.subplots(2, 3, figsize=(20, 15))
        for layer, (ax, layer_logs) in enumerate(zip(axs.flatten(), self.layer_logs)):
            train_data = [(log['step'], log['loss']) for log in layer_logs if 'loss' in log]
            eval_data = [(log['step'], log['eval_loss']) for log in layer_logs if 'eval_loss' in log]
            
            if train_data:
                steps, losses = zip(*train_data)
                ax.plot(steps, losses, label='Train Loss')
            if eval_data:
                steps, losses = zip(*eval_data)
                ax.plot(steps, losses, label='Validation Loss')
            
            ax.set_title(f'Layer {layer} Loss')
            ax.set_xlabel('Steps')
            ax.set_ylabel('Loss')
            ax.legend()
        
        plt.tight_layout()
        plt.savefig('hiddenstate_cosineloss_layerloss.png')
        plt.close()
        




    

2024-09-21 23:41:51.900069: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-21 23:41:51.911890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 23:41:51.924491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 23:41:51.928283: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 23:41:51.938977: I tensorflow/core/platform/cpu_feature_guar

In [5]:
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification, DistilBertConfig, DistilBertForMaskedLM, DataCollatorForLanguageModeling
from iDistilbert import iDistilBertForMaskedLM
from transformers import AutoTokenizer

student_id = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(student_id)
#Load Models

teacher_id = "distilbert/distilbert-base-uncased"
teacher_config = DistilBertConfig(    
    distance_metric = "cosine_distance",
    activation_function = "softmax",
    signed_inhibitor =  False,
    alpha = 0,
    center = False,
    output_contexts = False,
    output_hidden_states = True,
)
    
teacher_model = iDistilBertForMaskedLM.from_pretrained(
        teacher_id,
        config=teacher_config,
    )

student_config = DistilBertConfig(
    distance_metric = "manhattan_distance",
    activation_function = "relu",
    signed_inhibitor =  True,
    alpha = 0,
    center = True,
    output_contexts = False,
    output_hidden_states = True,
    )

student_model = iDistilBertForMaskedLM(student_config)

initialized_weights = torch.load('/shared/Tony/MSc2024/distilbert_init/models/qk_inhibitor_init.pth')
student_model.load_state_dict(initialized_weights, strict=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model.to(device)
student_model.to(device)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_2710843/1013190556.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode

iDistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): iDistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): iTransformer(
      (layer): ModuleList(
        (0-5): 6 x iTransformerBlock(
          (attention): iMultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout

In [6]:
EPOCHS = 2
BATCH_SIZE = 8
LEARNING_RATE = 5e-4
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LEARNING_RATE,
    logging_dir = './logs',
    load_best_model_at_end= True,
    metric_for_best_model="loss",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps = 268, 
    logging_steps = 20,
    save_steps=268,
    save_total_limit=2,
    seed = 42,
    #report_to=['comet_ml', 'tensorboard'],
    report_to=['tensorboard'],
    warmup_ratio=0.05,
    gradient_accumulation_steps=4,
    lr_scheduler_type="cosine",
)

trainer = distillTrainer(
    teacher_model=teacher_model,
    model=student_model,                         
    args=training_args,                  
    train_dataset=train_subset,         
    eval_dataset=val_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    hidden = True,
)


In [7]:
trainer.train()

Training layer group: transformer.layer.0
Current layer 0
Unfrozen parameters for layer 0:
  - distilbert.transformer.layer.0.attention.q_lin.weight
  - distilbert.transformer.layer.0.attention.q_lin.bias
  - distilbert.transformer.layer.0.attention.k_lin.weight
  - distilbert.transformer.layer.0.attention.k_lin.bias
  - distilbert.transformer.layer.0.attention.v_lin.weight
  - distilbert.transformer.layer.0.attention.v_lin.bias
  - distilbert.transformer.layer.0.attention.out_lin.weight
  - distilbert.transformer.layer.0.attention.out_lin.bias
  - distilbert.transformer.layer.0.sa_layer_norm.weight
  - distilbert.transformer.layer.0.sa_layer_norm.bias
  - distilbert.transformer.layer.0.ffn.lin1.weight
  - distilbert.transformer.layer.0.ffn.lin1.bias
  - distilbert.transformer.layer.0.ffn.lin2.weight
  - distilbert.transformer.layer.0.ffn.lin2.bias
  - distilbert.transformer.layer.0.output_layer_norm.weight
  - distilbert.transformer.layer.0.output_layer_norm.bias
7087872


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
268,0.066300,0.046045
536,0.049400,0.035637
804,0.043700,0.031754
1072,0.040100,0.028751
1340,0.035900,0.024684
1608,0.033000,0.022203
1876,0.031600,0.020833
2144,0.030400,0.020060
2412,0.029900,0.019680
2680,0.029200,0.019082


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

Training layer group: transformer.layer.1
Current layer 1
Unfrozen parameters for layer 1:
  - distilbert.transformer.layer.1.attention.q_lin.weight
  - distilbert.transformer.layer.1.attention.q_lin.bias
  - distilbert.transformer.layer.1.attention.k_lin.weight
  - distilbert.transformer.layer.1.attention.k_lin.bias
  - distilbert.transformer.layer.1.attention.v_lin.weight
  - distilbert.transformer.layer.1.attention.v_lin.bias
  - distilbert.transformer.layer.1.attention.out_lin.weight
  - distilbert.transformer.layer.1.attention.out_lin.bias
  - distilbert.transformer.layer.1.sa_layer_norm.weight
  - distilbert.transformer.layer.1.sa_layer_norm.bias
  - distilbert.transformer.layer.1.ffn.lin1.weight
  - distilbert.transformer.layer.1.ffn.lin1.bias
  - distilbert.transformer.layer.1.ffn.lin2.weight
  - distilbert.transformer.layer.1.ffn.lin2.bias
  - distilbert.transformer.layer.1.output_layer_norm.weight
  - distilbert.transformer.layer.1.output_layer_norm.bias
7087872


...

Training layer group: transformer.layer.5
Current layer 5
Unfrozen parameters for layer 5:
  - distilbert.transformer.layer.5.attention.q_lin.weight
  - distilbert.transformer.layer.5.attention.q_lin.bias
  - distilbert.transformer.layer.5.attention.k_lin.weight
  - distilbert.transformer.layer.5.attention.k_lin.bias
  - distilbert.transformer.layer.5.attention.v_lin.weight
  - distilbert.transformer.layer.5.attention.v_lin.bias
  - distilbert.transformer.layer.5.attention.out_lin.weight
  - distilbert.transformer.layer.5.attention.out_lin.bias
  - distilbert.transformer.layer.5.sa_layer_norm.weight
  - distilbert.transformer.layer.5.sa_layer_norm.bias
  - distilbert.transformer.layer.5.ffn.lin1.weight
  - distilbert.transformer.layer.5.ffn.lin1.bias
  - distilbert.transformer.layer.5.ffn.lin2.weight
  - distilbert.transformer.layer.5.ffn.lin2.bias
  - distilbert.transformer.layer.5.output_layer_norm.weight
  - distilbert.transformer.layer.5.output_layer_norm.bias
7087872


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
268,0.181100,0.162957
536,0.172400,0.156017
804,0.169900,0.153546
1072,0.168700,0.152380
1340,0.166700,0.150864
1608,0.165900,0.150582
1876,0.165300,0.149409
2144,0.163600,0.148962
2412,0.163800,0.148297
2680,0.163100,0.147246


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=2686, training_loss=0.17140709515036884, metrics={'train_runtime': 12625.2906, 'train_samples_per_second': 13.618, 'train_steps_per_second': 0.213, 'total_flos': 2.2787395294058496e+16, 'train_loss': 0.17140709515036884, 'epoch': 1.9996277684719894})

In [8]:
import os

folder = 'models/'
os.makedirs(folder, exist_ok=True)
torch.save(student_model.state_dict(), os.path.join(folder, 'hiddenstates2_inhibitor_init.pth'))

In [11]:
import matplotlib.pyplot as plt
import numpy as np

def smooth_data(data, window_size):
    cumsum = np.cumsum(np.insert(data, 0, 0)) 
    return (cumsum[window_size:] - cumsum[:-window_size]) / window_size

def organize_data_by_layer(data):
    organized_data = {}
    for entry in data:
        layer = entry['layer']
        if layer not in organized_data:
            organized_data[layer] = {'student': {'mean': [], 'max': [], 'min': []},
                                     'teacher': {'mean': [], 'max': [], 'min': []}}
        for model in ['student', 'teacher']:
            for stat in ['mean', 'max', 'min']:
                organized_data[layer][model][stat].append(entry[model][stat])
    return organized_data

def plot_all_layers(data, smooth_window=100):
    organized_data = organize_data_by_layer(data)
    num_layers = len(organized_data)
    
    fig, axs = plt.subplots(num_layers, 3, figsize=(20, 8 * num_layers))
    fig.suptitle('Layer Statistics Comparison (Smoothed)', fontsize=16)
    
    if num_layers == 1:
        axs = axs.reshape(1, -1)
    
    colors = {'student': 'blue', 'teacher': 'red'}
    stats = ['mean', 'max', 'min']
    
    for layer, (layer_num, layer_data) in enumerate(sorted(organized_data.items())):
        for col, stat in enumerate(stats):
            ax = axs[layer, col]
            for model in ['student', 'teacher']:
                original_data = layer_data[model][stat]
                
                # Plot original data with low alpha
                ax.plot(original_data, color=colors[model], alpha=0.3, linewidth=1)
                
                # Smooth and plot the data
                if len(original_data) > smooth_window:
                    smoothed_data = smooth_data(original_data, smooth_window)
                    ax.plot(range(smooth_window-1, len(original_data)), smoothed_data, 
                            color=colors[model], label=f'{model.capitalize()} (Smoothed)')
                else:
                    ax.plot(original_data, color=colors[model], label=model.capitalize())
            
            ax.set_title(f'Layer {layer_num} - {stat.capitalize()}', fontsize=12)
            ax.set_xlabel('Steps', fontsize=10)
            ax.set_ylabel('Value', fontsize=10)
            ax.legend(fontsize=8)
            ax.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('hiddenstates2_context_plot.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

In [12]:
plot_all_layers(trainer.context_loss_stats)